In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import myFunctions as mf
import importlib

In [2]:
importlib.reload(mf)

<module 'myFunctions' from '/home/jovyan/BasketballPlayoffs/myFunctions.py'>

In [7]:
plyrs = pickle.load(open( "allplayers.p", "rb" ))
plyrsreg = pickle.load(open("allplayersregular.p","rb"))
allshotspoff = pickle.load(open( "allshotspoff2017.p", "rb" ))
allshotsreg = pickle.load(open("allshotsreg2017.p","rb"))
PIDS = [i for i in plyrs.keys()]

In [8]:
allshotsreg

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,PlayerID
0,Shot Chart Detail,0021800002,177,203518,Alex Abrines,1610612760,Oklahoma City Thunder,1,0,41,...,Less Than 8 ft.,2,11,21,1,1,20181016,GSW,OKC,203518
1,Shot Chart Detail,0021800002,254,203518,Alex Abrines,1610612760,Oklahoma City Thunder,2,7,38,...,24+ ft.,25,-200,164,1,0,20181016,GSW,OKC,203518
2,Shot Chart Detail,0021800002,366,203518,Alex Abrines,1610612760,Oklahoma City Thunder,2,1,21,...,24+ ft.,26,-112,237,1,1,20181016,GSW,OKC,203518
3,Shot Chart Detail,0021800002,372,203518,Alex Abrines,1610612760,Oklahoma City Thunder,2,0,52,...,24+ ft.,26,92,245,1,0,20181016,GSW,OKC,203518
4,Shot Chart Detail,0021800002,383,203518,Alex Abrines,1610612760,Oklahoma City Thunder,2,0,4,...,8-16 ft.,14,90,111,1,0,20181016,GSW,OKC,203518
5,Shot Chart Detail,0021800002,535,203518,Alex Abrines,1610612760,Oklahoma City Thunder,3,1,21,...,24+ ft.,25,-226,110,1,0,20181016,GSW,OKC,203518
6,Shot Chart Detail,0021800002,553,203518,Alex Abrines,1610612760,Oklahoma City Thunder,3,0,24,...,24+ ft.,26,154,217,1,1,20181016,GSW,OKC,203518
7,Shot Chart Detail,0021800002,597,203518,Alex Abrines,1610612760,Oklahoma City Thunder,4,9,49,...,24+ ft.,22,-226,32,1,0,20181016,GSW,OKC,203518
8,Shot Chart Detail,0021800025,122,203518,Alex Abrines,1610612760,Oklahoma City Thunder,1,2,39,...,Less Than 8 ft.,2,-19,23,1,1,20181019,LAC,OKC,203518
9,Shot Chart Detail,0021800025,141,203518,Alex Abrines,1610612760,Oklahoma City Thunder,1,1,48,...,24+ ft.,24,143,204,1,1,20181019,LAC,OKC,203518


In [127]:
aggregation = {'SHOT_ATTEMPTED_FLAG': {'TOTAL_SHOTS': 'sum'}}

playOffTeams = allshotspoff.TEAM_NAME.unique()
playOffNames = allshotspoff.PLAYER_NAME.unique()
teamTotalReg = allshotsreg[allshotsreg.TEAM_NAME.isin(playOffTeams)].groupby(['GAME_ID','TEAM_NAME']).agg(aggregation)
teamTotalPoff = allshotspoff.groupby(['GAME_ID','TEAM_NAME']).agg(aggregation)
teamTotalReg.columns = teamTotalReg.columns.droplevel(0)

teamTotalPoff.columns = teamTotalPoff.columns.droplevel(0)
teamTotalReg

TOTAL_SHOTS
GAME_ID    TEAM_NAME                          
0021800001 Boston Celtics                   96
           Philadelphia 76ers               83
0021800002 Golden State Warriors            95
           Oklahoma City Thunder            87
0021800003 Milwaukee Bucks                  78
0021800004 Brooklyn Nets                    80
           Detroit Pistons                  88
0021800005 Indiana Pacers                   81
0021800006 Orlando Magic                    86
0021800008 Toronto Raptors                  92
0021800009 Houston Rockets                  92
0021800010 San Antonio Spurs                93
0021800011 Utah Jazz                        79
0021800012 Denver Nuggets                   87
           LA Clippers                      82
0021800014 Philadelphia 76ers               85
0021800016 Portland Trail Blazers          100
0021800017 Orlando Magic                    85
0021800018 Brooklyn Nets                    67
0021800019 Boston Celtics                   99
           Toronto Raptors                  92
0021800023 Indiana Pacers                   83
           Milwaukee Bucks                  87
0021800024 Golden State Warriors            87
           Utah Jazz                        89
0021800025 LA Clippers                      79
           Oklahoma City Thunder            94
0021800026 Brooklyn Nets                    71
           Indiana Pacers                   91
0021800027 Toronto Raptors                  88
...                                        ...
0021801203 Boston Celtics                   86
           Orlando Magic                    90
0021801205 Golden State Warriors            96
           LA Clippers                      55
0021801206 Denver Nuggets                   94
           Portland Trail Blazers           89
0021801208 Utah Jazz                        79
0021801210 Detroit Pistons                  83
0021801211 Boston Celtics                   83
0021801212 Philadelphia 76ers               75
0021801214 Toronto Raptors                  74
0021801215 Golden State Warriors            75
0021801217 Denver Nuggets                   87
           Utah Jazz                        76
0021801218 Houston Rockets                  88
           Oklahoma City Thunder            96
0021801219 Portland Trail Blazers           90
0021801220 Indiana Pacers                   82
0021801221 Brooklyn Nets                    90
0021801222 Orlando Magic                    88
0021801223 Detroit Pistons                  79
0021801224 Philadelphia 76ers               68
0021801225 Golden State Warriors            79
0021801226 Milwaukee Bucks                  84
           Oklahoma City Thunder            99
0021801227 San Antonio Spurs                85
0021801228 Denver Nuggets                   87
0021801229 LA Clippers                      82
           Utah Jazz                        67
0021801230 Portland Trail Blazers           51

[1312 rows x 1 columns]

11149

In [132]:
plyrsTotalReg = allshotsreg[np.logical_and(allshotsreg.TEAM_NAME.isin(playOffTeams),allshotsreg.PLAYER_NAME.isin(playOffNames))].groupby(['GAME_ID','TEAM_NAME','PLAYER_NAME']).agg(aggregation)
plyrsTotalPoff = allshotspoff.groupby(['GAME_ID','TEAM_NAME','PLAYER_NAME']).agg(aggregation)
plyrsTotalReg



plyrsTotalReg['TEAM_TOTAL_SHOTS'] = teamTotalReg.TOTAL_SHOTS
plyrsTotalReg['PCT_SHOTS'] = plyrsTotalReg.SHOT_ATTEMPTED_FLAG.TOTAL_SHOTS.values / plyrsTotalReg.TEAM_TOTAL_SHOTS.values

plyrsTotalReg


SHOT_ATTEMPTED_FLAG  \
                                                           TOTAL_SHOTS   
GAME_ID    TEAM_NAME              PLAYER_NAME                            
0021800001 Boston Celtics         Al Horford                         7   
                                  Aron Baynes                        7   
                                  Gordon Hayward                    12   
                                  Jaylen Brown                      13   
                                  Jayson Tatum                      17   
                                  Kyrie Irving                      14   
                                  Marcus Morris                     12   
                                  Marcus Smart                       4   
                                  Terry Rozier                      10   
           Philadelphia 76ers     Amir Johnson                       1   
                                  Ben Simmons                       14   
                                  JJ Redick                         17   
                                  Joel Embiid                       21   
                                  T.J. McConnell                     5   
0021800002 Golden State Warriors  Alfonzo McKinnie                   1   
                                  Andre Iguodala                     2   
                                  Damian Jones                       7   
                                  Draymond Green                     6   
                                  Kevin Durant                      21   
                                  Kevon Looney                      11   
                                  Klay Thompson                     20   
                                  Quinn Cook                         2   
                                  Shaun Livingston                   5   
                                  Stephen Curry                     20   
           Oklahoma City Thunder  Dennis Schroder                   19   
                                  Jerami Grant                       7   
                                  Nerlens Noel                       2   
                                  Paul George                       23   
                                  Raymond Felton                     5   
                                  Steven Adams                      12   
...                                                                ...   
0021801227 San Antonio Spurs      Patty Mills                        5   
                                  Rudy Gay                          10   
0021801228 Denver Nuggets         Gary Harris                       11   
                                  Jamal Murray                      12   
                                  Malik Beasley                     10   
                                  Mason Plumlee                      2   
                                  Monte Morris                       8   
                                  Nikola Jokic                      26   
                                  Paul Millsap                       7   
                                  Torrey Craig                       1   
                                  Will Barton                       10   
0021801229 LA Clippers            Danilo Gallinari                  11   
                                  Garrett Temple                     4   
                                  Ivica Zubac                       13   
                                  JaMychal Green                     9   
                                  Lou Williams                      17   
                                  Montrezl Harrell                  13   
                                  Patrick Beverley                   9   
                                  Tyrone Wallace                     3   
                                  Wilson Chandler                    3   
           Utah Jazz              Ekpe Udoh            

In [134]:
plyrsTotalPoff['TEAM_TOTAL_SHOTS'] = teamTotalPoff.TOTAL_SHOTS
plyrsTotalPoff['PCT_SHOTS'] = plyrsTotalPoff.SHOT_ATTEMPTED_FLAG.TOTAL_SHOTS.values / plyrsTotalPoff.TEAM_TOTAL_SHOTS.values

plyrsTotalPoff

1504

In [138]:
len(meanPctPoff)

183

In [143]:
##
meanPctReg = plyrsTotalReg.groupby('PLAYER_NAME').mean()
meanPctPoff = plyrsTotalPoff.groupby('PLAYER_NAME').mean()
meanPctReg['PCT_DIFF'] = np.abs(meanPctReg.PCT_SHOTS.values - meanPctPoff.PCT_SHOTS.values)
print(np.median(meanPctReg.PCT_DIFF.values))
print(np.mean(meanPctReg.PCT_DIFF.values))
print(np.max(meanPctReg.PCT_DIFF.values))

0.018317087560465437
0.021311709955450824
0.10454706494032337
